In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-11-26 19:12:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.09s   

2022-11-26 19:12:43 (10.1 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Load Amazon Review data into spark dataframe
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [6]:
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RED72VWWCOS7S|          1|            2|          8|   N|                Y|
| RZHWQ208LTEPV|          5|            0|          0|   N|                Y|
|R37LBC3XAVLYOO|          5|            4|          5|   N|                Y|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|                Y|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|                Y|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|                Y|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|                Y|
|R22C8FMBSTFRY8|          5|            2|          2|   N|                Y|
|R118NNIQ75XPGO|          3|            0|          0|   N|                Y|
|R30HYXHZQ49621|          2|            0|          0|   N|     

In [7]:
# Filter vine table to reviews with greater than 20 total votes
vine_filtered = vine_df.filter(vine_df["total_votes"] > 20)

In [8]:
vine_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|     

In [9]:
vine_filtered = vine_filtered.withColumn("help_rate", (vine_filtered['helpful_votes'] / vine_filtered['total_votes']))
vine_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         help_rate|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|0.8275862068965517|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|0.9166666666666666|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|0.9666666666666667|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|0.2857142857142857|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|0.8928571428571429|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|               1.0|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|0.8333333333333334|
|  RLNULBKRWNNR|          5|  

In [10]:
vine_filtered.count()

49209

In [11]:
vine_helpful = vine_filtered.filter(vine_filtered['help_rate']>0.5)
vine_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         help_rate|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|0.8275862068965517|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|0.9166666666666666|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|0.9666666666666667|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|0.8928571428571429|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|               1.0|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|0.8333333333333334|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|0.9767441860465116|
| R9QNQUL94RX1F|          3|  

In [12]:
vine_helpful.count()

46014

In [13]:
paid_reviews = vine_helpful.filter(vine_helpful['vine']=="Y")
paid_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         help_rate|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R28DXTC3JQ9IY1|          4|           24|         26|   Y|                N|0.9230769230769231|
|R3AFZKLQXATHBU|          5|           44|         49|   Y|                N|0.8979591836734694|
|R2RUUF2JPJPC0E|          4|           20|         22|   Y|                N|0.9090909090909091|
| RFZ2WUH4248AB|          2|           26|         27|   Y|                N|0.9629629629629629|
|R1Q4LVHIFOWYFR|          5|           23|         28|   Y|                N|0.8214285714285714|
|R2Z7C8YCRSC9DP|          5|           22|         22|   Y|                N|               1.0|
|R3J8OI5CB74P5K|          1|           22|         25|   Y|                N|              0.88|
| RH39LMKN6AZDC|          5|  

In [14]:
paid_reviews.count()

373

In [15]:
unpaid_reviews = vine_helpful.filter(vine_helpful['vine']=="N")
unpaid_reviews.show()
unpaid_reviews.count()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         help_rate|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|0.8275862068965517|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|0.9166666666666666|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|0.9666666666666667|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|0.8928571428571429|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|               1.0|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|0.8333333333333334|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|0.9767441860465116|
| R9QNQUL94RX1F|          3|  

45641

In [21]:
import pyspark.sql.functions as F

paid_stars = paid_reviews.groupBy("star_rating").agg(
    (F.count("review_id")).alias("count"),
    (F.count("review_id") / paid_reviews.count()).alias('percentage')
).orderBy("star_rating").show()

+-----------+-----+--------------------+
|star_rating|count|          percentage|
+-----------+-----+--------------------+
|          1|    8|0.021447721179624665|
|          2|   11|0.029490616621983913|
|          3|   59|  0.1581769436997319|
|          4|  124|  0.3324396782841823|
|          5|  171|  0.4584450402144772|
+-----------+-----+--------------------+



In [22]:
unpaid_stars = unpaid_reviews.groupBy("star_rating").agg(
    (F.count("review_id")).alias("count"),
    (F.count("review_id") / unpaid_reviews.count()).alias('percentage')
).orderBy("star_rating").show()

+-----------+-----+--------------------+
|star_rating|count|          percentage|
+-----------+-----+--------------------+
|          1| 9507| 0.20829955522446922|
|          2| 2795|0.061238798448763176|
|          3| 3882| 0.08505510396354156|
|          4| 6919| 0.15159615258210818|
|          5|22538| 0.49381038978111785|
+-----------+-----+--------------------+

